In [ ]:
!pip install huggingface_hub
#You already have it if you installed transformers or datasets

!huggingface-cli login
#Log in using a token from huggingface.co/settings/tokens
#Create a model or dataset repo from the CLI if needed
!huggingface-cli repo create repo_name --type {model, dataset, space}

Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 5, in <module>
    from huggingface_hub.commands.huggingface_cli import main
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 18, in <module>
    from huggingface_hub.commands.download import DownloadCommand
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py", line 44, in <module>
    from huggingface_hub._snapshot_download import snapshot_download
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/_snapshot_download.py", line 17, in <module>
    from .hf_api import DatasetInfo, HfApi, ModelInfo, SpaceInfo
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py", line 61, in <module>
    from ._inference_endpoints import InferenceEndpoint, InferenceEndpointType
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/_inference_endpoints.py", line 9, in <module>
    from .inference._cli

In [ ]:
from datasets import load_dataset

ds = load_dataset("josephnashat/ArabicSentimentAnalysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

ds['train'].to_pandas().head(10)


,text,polarity
0,المكان الذي يمكنك فيه مراجعة الذات والتفكر هو ...,1
1,موقع رائع وحديقة رائعة ويستحق نجمةّ إضافية \n...,0
2,أسوأ فندق أقمت فيه على الإطلاق \nيستغرق تسجيل...,-1
3,بدون روح كأنه فندق ثلاثة نجوم \nبدون إدارة اح...,0
4,فندق جميل مع سوء الإدارة والخدمات. \nمن الخار...,-1
5,فندق جميل، منظر رائع من بركة السباحة (على السط...,1
6,مكان لطيف وهادئ بعيدًا عن الزحام \nعثرت على ه...,1
7,"راقي و كلاسيكي"" \nفندق كبير مصمم بفن الديكور ...",1
8,تحذير للرجال الذي يحبون تناول بعض البيرة ليلاً...,1
9,فندق رائع \nبعد 4 أيام قضيناها في ليسبوا، وصل...,1


In [ ]:
print(f"Number of splits/sets in the dataset: {len(ds)}")


Number of splits/sets in the dataset: 1


In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
ds = load_dataset("josephnashat/ArabicSentimentAnalysis")

# Describe the dataset
print("Dataset Description:")
print(ds)

# Print some basic information about the dataset
print("\nDataset Splits:")
for split in ds:
  print(f"- {split}: {ds[split].shape[0]} examples")

print("\nDataset Features:")
print(ds['train'].features)

# Show the first 10 rows of the training set as a Pandas DataFrame
print("\nFirst 10 rows of the training set:")
print(ds['train'].to_pandas().head(10))



Dataset Description:
DatasetDict({
    train: Dataset({
        features: ['text', 'polarity'],
        num_rows: 964
    })
})

Dataset Splits:
- train: 964 examples

Dataset Features:
{'text': Value(dtype='string', id=None), 'polarity': Value(dtype='int64', id=None)}

First 10 rows of the training set:
                                                text  polarity
0  المكان الذي يمكنك فيه مراجعة الذات والتفكر هو ...         1
1  موقع رائع وحديقة رائعة ويستحق نجمةّ إضافية  \n...         0
2  أسوأ فندق أقمت فيه على الإطلاق  \nيستغرق تسجيل...        -1
3  بدون روح كأنه فندق ثلاثة نجوم  \nبدون إدارة اح...         0
4  فندق جميل مع سوء الإدارة والخدمات.  \nمن الخار...        -1
5  فندق جميل، منظر رائع من بركة السباحة (على السط...         1
6  مكان لطيف وهادئ بعيدًا عن الزحام  \nعثرت على ه...         1
7  راقي و كلاسيكي"  \nفندق كبير مصمم بفن الديكور ...         1
8  تحذير للرجال الذي يحبون تناول بعض البيرة ليلاً...         1
9  فندق رائع  \nبعد 4 أيام قضيناها في ليسبوا، وصل...         1


In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

def preprocess_arabic_text(text):
  """
  Preprocesses Arabic text by removing diacritics, punctuation, and stop words, and stemming words.
  """
  # Remove diacritics (harakat)
  text = re.sub(r'[^\w\s]', '', text)

  # Tokenize the text into words
  tokens = word_tokenize(text)

  # Stem the words using ISRIStemmer
  stemmer = ISRIStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]

  # Remove stop words (optional, you can define your own stop words list)
  stop_words = set(stopwords.words('arabic'))
  stemmed_tokens = [token for token in stemmed_tokens if token not in stop_words]


  # Join the processed tokens back into a string
  processed_text = " ".join(stemmed_tokens)

  return processed_text

# Apply the preprocessing function to the 'text' column of the training set
ds['train'] = ds['train'].map(lambda example: {'text': preprocess_arabic_text(example['text'])})


# Show the first 3 rows of the training set with preprocessed text
print("\nFirst 3 rows of the training set with preprocessed text:")
print(ds['train'].to_pandas().head(3))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Map:   0%|          | 0/964 [00:00<?, ? examples/s]


First 3 rows of the training set with preprocessed text:
                                                text  polarity
0  يمك فكر كوكروبيت غنا كثر زير غنا رغم فقر لحظ ط...         1
1  وقع رئع حدق رئع سحق نجم ضفي رغم ان غرف فخر فهي...         0
2  اسأ ندق اقم طلق غرق سجل وصل حلي 30 دقق بهظ ثمن...        -1


In [ ]:
!pip install pyspellchecker

from spellchecker import SpellChecker

spell = SpellChecker(language='ar')

def correct_spelling(text):
  words = text.split()
  corrected_words = [spell.correction(word) if spell.correction(word) is not None else word for word in words]
  return " ".join(corrected_words)


# Apply the spelling correction function to the 'text' column of the training set
ds['train'] = ds['train'].map(lambda example: {'text': correct_spelling(example['text'])})

# Show the first 3 rows of the training set with corrected text
print("\nFirst 3 rows of the training set with corrected text:")
print(ds['train'].to_pandas().head(3))


Map:   0%|          | 0/964 [00:00<?, ? examples/s]


First 3 rows of the training set with corrected text:
                                                text  polarity
0  يمك فكر كوربيت أنا كثر زير أنا رغم فقر لحظ طرق...         1
1  وقع رئع حدق رئع سحق نجم في رغم ان غرف فخر فهي ...         0
2  اسم ندق اقم طلق غرق سجل وصل حلي 30 دقق به ثمن ...        -1


In [ ]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

# Assuming 'ds' is your dataset and 'polarity' is the column containing the labels
label_counts = Counter(ds['train']['polarity'])
print("Label distribution before balancing:")
print(label_counts)

# Separate features (X) and labels (y)
X = pd.DataFrame(ds['train'])[['text']]  # Replace 'text' with your feature column
y = ds['train']['polarity']

# Initialize the RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Apply oversampling to the dataset
X_resampled, y_resampled = ros.fit_resample(X, y)

# Create a new dataset with the resampled data
resampled_ds = pd.concat([X_resampled, pd.DataFrame({'polarity': y_resampled})], axis=1)

# Check the label distribution after oversampling
label_counts_resampled = Counter(resampled_ds['polarity'])
print("\nLabel distribution after oversampling:")
print(label_counts_resampled)


Label distribution before balancing:
Counter({1: 738, -1: 114, 0: 112})

Label distribution after oversampling:
Counter({1: 738, 0: 738, -1: 738})


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    resampled_ds['text'], resampled_ds['polarity'], test_size=0.2, random_state=42
)

# Create a TF-IDF vectorizer to convert text data into numerical features
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)  # You can adjust max_iter
model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.9525959367945824

Classification Report:
              precision    recall  f1-score   support

          -1       0.99      1.00      0.99       144
           0       0.90      0.99      0.94       148
           1       0.99      0.87      0.93       151

    accuracy                           0.95       443
   macro avg       0.96      0.95      0.95       443
weighted avg       0.96      0.95      0.95       443



In [ ]:
import pandas as pd
# Create a DataFrame to compare actual and predicted labels
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

# Show the first 20 rows of the results
print(results_df.head(10))

      Actual  Predicted
1466      -1         -1
1418      -1         -1
581        1          1
1517      -1         -1
1231      -1         -1
1518      -1         -1
1456      -1         -1
544        1          1
1931       0          0
1658       0          0
